In [1]:
from dotenv import load_dotenv

load_dotenv()

True

먼저 벡터스토어에 적용할 임의의 문서를 로드해보겠습니다.<br>

langchain_core.documents.Document는 page_content, metadata를 입력으로 받을 수 있습니다.<br>
예제에서는 몇 가지 다른 메타데이터를 가진 문서를 정의했습니다.<br>

이 클래스 외에도 다양한 방법으로 문서를 로드할 수 있습니다.

In [2]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

# Vector Store

구조화되지 않은 텍스트 등을 저장하고 검색하는 일반적인 방법으로는 이 텍스트를 그대로 사용하는 것이 아닌 임베딩 방법을 통해 벡터화하여 유사도 분석 등을 통해 검색을 하는 방법이 있습니다.<br>

LangChain VectorStore 객체는 Documnet 객체나 텍스트 등을 스토어에 추가하고 다양한 유사도 분석 메트릭을 사용하여 쿼리하는 방법을 가지고 있습니다.<br>

LangChain은 다양한 벡터 스토어를 포함했으며, 클라우드에서 사용하는 벡터 스토어 등도 포함되어 있습니다.
랭체인에서 사용할 수 있는 다양한 벡터스토어는 아래 링크에서 확인할 수 있습니다.<br>
https://python.langchain.com/v0.2/docs/integrations/vectorstores/

여기에서는 메모리 내 구현을 포함하는 Chroma를 사용하여 LangChain VectorStores의 사용 방법을 확인해보도록 하겠습니다.<br>

벡터스토어를 사용하기 위해서는 텍스트를 임베딩 벡터로 변환하기 위한 임베딩 모델이 필요한데요. LangChain은 이러한 임베딩 모델도 다양하게 포함했습니다. 사용할 수 있는 임베딩들의 링크는 다음과 같습니다.<br>
https://python.langchain.com/v0.2/docs/integrations/text_embedding/

여기서는 OpenAI 임베딩을 사용하여 구현해보도록 하겠습니다.

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents, # 정의해둔 문서를 사용
    embedding=OpenAIEmbeddings(), # OpenAI 임베딩 사용
)

.from_documents 메서드를 통해 문서를 벡터스토어에 등록할 수 있습니다.<br>
인스턴스화 한 후에도 문서를 추가 등록하는 등의 다양한 방법은 아래 링크에서 확인할 수 있습니다.<br>
https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document

이렇게 문서가 포함된 벡터스토어를 인스턴스화하면 쿼리를 수행할 수 있으며 다음과 같은 쿼리 방법을 포함합니다.<br>

- 동기식 및 비동기식
- 문자열 쿼리별, 벡터별
- 유사성 점수 반환 여부
- 유사성 및 최대 한계 관련성 파라미터 설정

메서드는 일반적으로 출력에 문서 객체 목록을 포함합니다.

## Examples

벡터스토어에 등록된 문서에 대해 유사도를 쿼리해보도록 하겠습니다

In [4]:
vectorstore.similarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

비동기식으로도 쿼리할 수 있습니다.

In [5]:
await vectorstore.asimilarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

유사도 점수도 쿼리할 수 있습니다

In [6]:
# Note that providers implement different scores; Chroma here
# returns a distance metric that should vary inversely with
# similarity.

vectorstore.similarity_search_with_score("cat")

[(Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  0.375326931476593),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  0.4833090305328369),
 (Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
  0.4958883225917816),
 (Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
  0.4974174499511719)]

임베딩 간 비교를 해보면 동일하다는 걸 확인할 수 있습니다.

In [7]:
embedding = OpenAIEmbeddings().embed_query("cat")
vectorstore.similarity_search_by_vector(embedding)

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

벡터스토어에 대한 추가적인 자료는 공식 페이지에서 확인할 수 있습니다.

https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html
https://python.langchain.com/v0.2/docs/how_to/vectorstores/


## 추가 예제

공식 문서 외에 몇 가지 예제를 확인해보도록 하겠습니다.<br>

벡터스토어를 사용한 서비스를 진행하다보면 중간에 문서를 추가해야하는 상황이 생기거나,<br>
잠깐 벡터스토어를 셧다운 한 후에 다시 로드해야 할 경우가 있을 것 같다는 생각이 드는데요. 
이에 대해 알아보도록 하겠습니다.

### 벡터스토어에 문서 추가하기

먼저 텍스트를 추가하는 방법입니다. 기존에는 Documents 클래스를 사용해서 문서를 만들었지만 저는 귀찮아서 add_text를 해보도록 하겠습니다.<br>
동기적으로 추가하고자 add_texts를 사용해보았습니다.<br>
add_texts, add_documents 뿐만 아니라 비동기 설정이 가능한 메서드등을 사용할 수 있습니다.

In [8]:
vectorstore.add_texts(["Shrimps are so delicious"])

['ba012058-40a3-4768-a126-48865b529e53']

In [9]:
await vectorstore.asimilarity_search("shrimp")

[Document(page_content='Shrimps are so delicious'),
 Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'})]

### 벡터스토어 저장 및 로드하기

이번에는 벡터스토어를 잠깐 저장해두고 다시 로드할 수 있는지 테스트해보도록 하겠습니다.<br>
예시로 사용하고 있는 벡터스토어는 Chroma로 벡터스토어마다 방법이 다르니 쓰시는 구조에 맞춰 공식 문서에서 검색하셔야 합니다.<br>

Chroma의 경우 맨 처음 벡터스토어를 설정할 때 persist_directory를 설정하여 벡터스토어를 저장할 수 있습니다.

In [10]:
# 실행 때 지정해야 저장됨
vectorstore = Chroma.from_documents(
    documents, # 정의해둔 문서를 사용
    embedding=OpenAIEmbeddings(), # OpenAI 임베딩 사용
    persist_directory="./chroma_db") # 저장 경로 설정

In [11]:
# add_text한 것이기 때문에 shrimp 관련 내용이나 업데이트, 딜리트한 내용은 사라짐
await vectorstore.asimilarity_search("shrimp")

[Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

In [12]:
# 다시 애드해둔다면?
vectorstore.add_texts(["Shrimps are so delicious"])

['dcf2b41c-5828-49d6-a977-853d9abc22ee']

저장된 벡터스토어는 다음과 같이 로드할 수 있습니다.

In [13]:
vectorstore_load = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())
# 다시 애드하면 다행히 있다!
await vectorstore_load.asimilarity_search("shrimp")

[Document(page_content='Shrimps are so delicious'),
 Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'})]